In [1]:
!pip install pennylane
!pip isntall torch

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.1/56.1 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 15.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 930.0/930.0 kB 19.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 28.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 17.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 167.9/167.9 kB 10.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.6/8.6 MB 27.4 MB/s eta 0:00:00
ERROR: unknown command "isntall" - maybe you meant "install"


In [422]:
import numpy as np

def add_zero_row_col(matrix, m):
    """
    n x n 행렬에 m번째 열과 행을 추가하고 0으로 채워서 (n+1) x (n+1) 크기의 행렬 생성.

    Args:
        matrix (np.array): 원본 n x n 행렬
        m (int): 추가할 행과 열의 위치 (0-based index)

    Returns:
        np.array: 수정된 (n+1) x (n+1) 행렬
    """
    n = matrix.shape[0]

    # 새로운 (n+1)x(n+1) 크기의 행렬 생성 및 기존 값 복사
    new_matrix = np.zeros((n + 1, n + 1))

    # 원래 행렬 값 복사
    new_matrix[:m, :m] = matrix[:m, :m]  # 상단 좌측
    new_matrix[:m, m+1:] = matrix[:m, m:]  # 상단 우측
    new_matrix[m+1:, :m] = matrix[m:, :m]  # 하단 좌측
    new_matrix[m+1:, m+1:] = matrix[m:, m:]  # 하단 우측

    return new_matrix




In [423]:
import numpy as np

def symmetric_matrix_upper_to_lower(matrix):
    """
    대각 위쪽에만 값이 있는 행렬을 받아, 대칭적으로 대각 아래에 복사하는 함수.

    Parameters:
        matrix (np.ndarray): 상삼각 행렬(대각 위에만 값이 있는 행렬)

    Returns:
        np.ndarray: 대각 아래에 값이 복사된 대칭 행렬
    """
    if not isinstance(matrix, np.ndarray):
        raise ValueError("입력은 numpy.ndarray 타입이어야 합니다.")

    if matrix.shape[0] != matrix.shape[1]:
        raise ValueError("입력 행렬은 정사각 행렬이어야 합니다.")

    symmetric_matrix = matrix.copy()

    # 대각 아래쪽에 대칭적으로 복사
    i_lower = np.tril_indices_from(symmetric_matrix, k=-1)
    symmetric_matrix[i_lower] = symmetric_matrix.T[i_lower]

    return symmetric_matrix





In [424]:
def remove_zero_diagonal_nodes(matrix):
    """
    행렬에서 대각선의 행과 열이 모두 0인 경우 해당 행과 열을 제거.

    Args:
        matrix (np.array): 초기 해밀토니안 행렬

    Returns:
        np.array: 축소된 해밀토니안 행렬
    """
    n = matrix.shape[0]

    # 제거할 인덱스를 추적
    keep_indices = []
    for i in range(n):
        row_sum = np.sum(matrix[i, :])  # i번째 행의 합
        col_sum = np.sum(matrix[:, i])  # i번째 열의 합

        # 대각 원소에 해당하는 행과 열이 모두 0이 아니면 유지
        if row_sum != 0 or col_sum != 0:
            keep_indices.append(i)

    # 행과 열이 모두 0인 노드를 제거하고 새로운 행렬 생성
    reduced_matrix = matrix[np.ix_(keep_indices, keep_indices)]

    return reduced_matrix
def zero_lower_triangle(matrix):
    """
    주어진 numpy 행렬의 하삼각 요소(대각선 아래)를 0으로 만드는 함수.

    Args:
        matrix (np.array): 입력 행렬

    Returns:
        np.array: 하삼각이 0으로 설정된 행렬
    """
    result = np.copy(matrix)
    rows, cols = result.shape

    for i in range(rows):
        for j in range(i):
            result[i, j] = 0

    return result
def reduce_hamiltonian(J,k, l,node_assignments, sign):
    """
    주어진 제약 조건 Z_k = sign * Z_l을 적용하여 해밀토니안을 축소하는 함수

    Args:
    - J (np.array): 초기 해밀토니안 행렬 (대각 항 포함)
    - k (int): 제거할 변수 인덱스
    - l (int): 대체할 변수 인덱스
    - sign (int): 관계 (1이면 동일, -1이면 반대)

    Returns:
    - np.array: 축소된 해밀토니안 행렬
    """
    # 새로운 상호작용 업데이트: J[i,l] = J[i,l] + sign * J[i,k]

    #J = symmetric_matrix_upper_to_lower(J)
    for i in range(J.shape[0]):
        if i != k and i != l:
            J[i, l] += sign * J[i, k]
            J[l, i] += sign * J[k, i]
    # 대각선 업데이트 (자기 항 Z_k = sign * Z_l 적용)
    J[l, l] = sign * J[k, k] + J[l, l]  # 자기 항 반영
    J = zero_lower_triangle(J)
    key_list = sorted(node_assignments.keys())



    J[:,k] = 0
    J[k,:] = 0
    # 변수 제거 (행 및 열 삭제)
    J = np.delete(J, k, axis=0)

    # 열 제거
    J = np.delete(J, k, axis=1)
    R = copy.deepcopy(J)
    for key in key_list:
      R = add_zero_row_col(R,key)
    return J,R





In [425]:

def signed_softmax_rewards(rewards, beta=15.0):
    """
    Apply softmax transformation to absolute values of rewards
    while preserving their original sign.

    Args:
        rewards (np.ndarray): Array of reward values.
        beta (float): Temperature parameter to control sharpness of softmax.

    Returns:
        np.ndarray: Transformed reward values with preserved sign.
    """
    rewards = np.array(rewards)

    # Step 1: Compute absolute values and apply softmax
    abs_rewards = np.abs(rewards)
    scaled_rewards = beta * abs_rewards
    exp_rewards = np.exp(scaled_rewards - np.max(scaled_rewards))  # Numerical stability
    softmax_vals = exp_rewards / np.sum(exp_rewards)

    # Step 2: Restore original sign
    signed_rewards = np.sign(rewards) * softmax_vals
    return signed_rewards

In [426]:
from itertools import product

def find_valid_combinations(n, same_constraints, diff_constraints):
    """
    주어진 조건을 만족하는 가능한 조합을 찾는 함수.

    Args:
        n (int): 변수의 개수.
        same_constraints (list of tuples): 같은 값을 가져야 하는 쌍 (e.g. [(1, 2), (3, 4)]).
        diff_constraints (list of tuples): 다른 값을 가져야 하는 쌍 (e.g. [(2, 3), (4, 5)]).

    Returns:
        list of list: 모든 가능한 조합
    """
    valid_solutions = []

    # 가능한 모든 2^n 조합 생성 (각 변수는 -1 또는 1을 가짐)
    for candidate in product([-1, 1], repeat=n):
        assignment = list(candidate)
        is_valid = True

        # 같은 값을 가져야 하는 조건 확인
        for (i, j) in same_constraints:
            if assignment[i] != assignment[j]:
                is_valid = False
                break

        # 다른 값을 가져야 하는 조건 확인
        for (i, j) in diff_constraints:
            if assignment[i] == assignment[j]:
                is_valid = False
                break

        if is_valid:
            valid_solutions.append(assignment)

    return valid_solutions




In [427]:
import copy
class Edge:
  def __init__(self,edge_type ,value):
    self.value =value
    if edge_type =="same":
      self.same = True
    else:
      self.same = False

def dfs(node, node_number, edge_list, tmp_list, result_list):
  if node==node_number:
    result_list.append(copy.deepcopy(tmp_list))
    return 0

  if len(edge_list[node]) ==0 :
    tmp_list[node] =1
    dfs(node+1, node_number, edge_list, tmp_list, result_list)
    tmp_list[node] =-1
    dfs(node+1, node_number, edge_list, tmp_list, result_list)
    return 0

  tmp_list[node] =0
  for e in edge_list[node]:
    # print(node ,e.value ,tmp_list, e.value ,tmp_list[e.value] ,e.same)
    if e.same:
      res= tmp_list[e.value]

    if not e.same:
      res= -1*tmp_list[e.value]

    if(tmp_list[node] != 0 and tmp_list[node] != res ):
        # print("chk")
        return -1

    tmp_list[node] =res
  dfs(node+1, node_number, edge_list, tmp_list ,result_list)
  return



from itertools import product
def brute_force_valid_combinations(n, same_constraints, diff_constraints):
    """
    Brute-force 방법을 사용하여 조건을 만족하는 가능한 조합을 찾는 함수.

    Args:
        n (int): 변수의 개수.
        same_constraints (list of tuples): 같은 값을 가져야 하는 쌍.
        diff_constraints (list of tuples): 다른 값을 가져야 하는 쌍.

    Returns:
        list of list: 모든 가능한 조합.
    """
    valid_solutions = []

    # 가능한 모든 조합 생성 (-1, 1)
    cout =0
    for candidate in product([-1, 1], repeat=n):
        cout += 1
        assignment = list(candidate)
        is_valid = True

        # 같은 값 조건 확인
        for (i, j) in same_constraints:
            if assignment[i] != assignment[j]:
                is_valid = False
                break

        # 다른 값 조건 확인
        for (i, j) in diff_constraints:
            if assignment[i] == assignment[j]:
                is_valid = False
                break

        if is_valid:
            valid_solutions.append(assignment)
    return sorted(valid_solutions)

def get_case(same_list, diff_list, node_number):
  edge_list ={}
  for i in range(0,node_number):
    edge_list[i] = []

  for e in same_list:
    if e[0] < e[1]:
      edge_list[e[1]].append(Edge('same', e[0]))
    else:
      edge_list[e[0]].append(Edge('same', e[1]))


  for e in diff_list:
    if e[0] < e[1]:
      edge_list[e[1]].append(Edge('diff', e[0]))
    else:
      edge_list[e[0]].append(Edge('diff', e[1]))


  tmp_list = [0]*node_number
  result_list =[]
  dfs(0 ,node_number, edge_list,tmp_list, result_list)
  # for res in result_list:
    # print(res)

  return result_list


In [428]:
import numpy as np

class AdamOptimizer:
    def __init__(self, lr=0.001, beta1=0.9, beta2=0.999, epsilon=1e-8):
        """
        Adam optimizer 초기화

        Parameters:
        - lr: 학습률 (learning rate)
        - beta1: 1차 모멘트 추정의 감쇠 계수
        - beta2: 2차 모멘트 추정의 감쇠 계수
        - epsilon: 수치적 안정성을 위한 작은 값
        """
        self.lr = lr
        self.beta1 = beta1
        self.beta2 = beta2
        self.epsilon = epsilon
        self.m = None  # 1차 모멘트 (평균)
        self.v = None  # 2차 모멘트 (분산)
        self.t = 0  # 시간 스텝

    def step(self, gradient, params):
        """
        Adam 최적화 단계 수행

        Parameters:
        - gradient: 현재의 그래디언트 값 (numpy 배열)
        - params: 현재의 파라미터 값 (numpy 배열)

        Returns:
        - 업데이트된 파라미터 값
        """
        if self.m is None:
            self.m = np.zeros_like(gradient)
            self.v = np.zeros_like(gradient)

        self.t += 1  # 시간 스텝 증가

        # 모멘트 업데이트
        self.m = self.beta1 * self.m + (1 - self.beta1) * gradient
        self.v = self.beta2 * self.v + (1 - self.beta2) * (gradient ** 2)

        # 편향 보정
        m_hat = self.m / (1 - self.beta1 ** self.t)
        v_hat = self.v / (1 - self.beta2 ** self.t)

        # 파라미터 업데이트
        update = self.lr * m_hat / (np.sqrt(v_hat) + self.epsilon)
        new_params = params - update

        return new_params


In [429]:
# 초기 파라미터와 그래디언트 예제
params = np.array([1.0, 2.0])
gradient = np.array([5.51, -5.52])

# Adam 최적화 객체 생성
adam = AdamOptimizer(lr=0.01)

# 업데이트 수행
new_params = adam.step(gradient, params)

print("Updated Parameters:", new_params)


Updated Parameters: [0.99 2.01]


In [430]:

# 업데이트 수행
new_params = adam.step(gradient, new_params)

print("Updated Parameters:", new_params)

Updated Parameters: [0.98 2.02]


In [431]:
import pennylane as qml  # Importing PennyLane for quantum computing
from pennylane import numpy as np  # Importing PennyLane's NumPy for compatibility
import torch  # Importing PyTorch for potential machine learning applications
import torch.nn as nn  # Importing PyTorch's neural network module
import copy  # Importing copy module for deep copying objects
import itertools  # Importing itertools for combinatorial operations
from tqdm import tqdm  # Importing tqdm for progress tracking
class AdamOptimizer():
	"""Stochastic gradient descent optimizer with Adam
	Note: All default values are from the original Adam paper
	Parameters
	----------
	params : list, length = len(coefs_) + len(intercepts_)
		The concatenated list containing coefs_ and intercepts_ in MLP model.
		Used for initializing velocities and updating params
	learning_rate_init : float, default=0.001
		The initial learning rate used. It controls the step-size in updating
		the weights
	beta_1 : float, default=0.9
		Exponential decay rate for estimates of first moment vector, should be
		in [0, 1)
	beta_2 : float, default=0.999
		Exponential decay rate for estimates of second moment vector, should be
		in [0, 1)
	epsilon : float, default=1e-8
		Value for numerical stability
	Attributes
	----------
	learning_rate : float
		The current learning rate
	t : int
		Timestep
	ms : list, length = len(params)
		First moment vectors
	vs : list, length = len(params)
		Second moment vectors
	References
	----------
	Kingma, Diederik, and Jimmy Ba.
	"Adam: A method for stochastic optimization."
	arXiv preprint arXiv:1412.6980 (2014).
	"""

	def __init__(self, params, learning_rate_init=0.001, beta_1=0.9,
				 beta_2=0.999, epsilon=1e-8, amsgrad = False):

		self.beta_1 = beta_1
		self.beta_2 = beta_2
		self.epsilon = epsilon
		if type(learning_rate_init) == float:
			self.learning_rate_init = np.ones(len(params))*learning_rate_init
		else:
			self.learning_rate_init = np.array(learning_rate_init)
		self.t = 0
		self.ms = [np.zeros_like(param) for param in params]
		self.vs = [np.zeros_like(param) for param in params]
		self.amsgrad = amsgrad
		self.max_vs = [np.zeros_like(param) for param in params]

	def get_updates(self, grads):
		"""Get the values used to update params with given gradients
		Parameters
		----------
		grads : list, length = len(coefs_) + len(intercepts_)
			Containing gradients with respect to coefs_ and intercepts_ in MLP
			model. So length should be aligned with params
		Returns
		-------
		updates : list, length = len(grads)
			The values to add to params
		"""
		self.t += 1
		self.ms = [self.beta_1 * m + (1 - self.beta_1) * grad
					 for m, grad in zip(self.ms, grads)]
		self.vs = [self.beta_2 * v + (1 - self.beta_2) * (grad ** 2)
					 for v, grad in zip(self.vs, grads)]
		self.max_vs = [np.maximum(v, max_v) for v, max_v in zip(self.vs, self.max_vs)]
		self.learning_rate = (self.learning_rate_init *
								np.sqrt(1 - self.beta_2 ** self.t) /
								(1 - self.beta_1 ** self.t))
		if self.amsgrad:
			updates = [lr * m / (np.sqrt(max_v) + self.epsilon)
			           for lr, m, max_v in zip(self.learning_rate, self.ms, self.max_vs)]
		else:
			updates = [lr * m / (np.sqrt(v) + self.epsilon)
					 for lr, m, v in zip(self.learning_rate, self.ms, self.vs)]
		return updates



class QAOA_layer():
    def __init__(self, depth, Q):
        """
        Initializes the QAOA layer with a given depth and QUBO matrix.

        Args:
            depth (int): The depth of the QAOA circuit (number of layers).
            Q (np.ndarray): QUBO matrix representing the quadratic unconstrained binary optimization problem.
        """
        self.Q = Q  # Store the QUBO matrix
        self.p = depth  # Store the QAOA depth
        self.ham = self.prepare_cost_hamiltonian()  # Prepare the cost Hamiltonian based on QUBO matrix
        self.dev = qml.device("default.qubit", wires=Q.shape[0])  # Define a quantum device with qubits equal to the size of Q

    def qaoa_circuit(self, params):
        """
        Constructs the QAOA circuit with the given parameters.

        Args:
            params (list): A list containing gamma and beta values for parameterized QAOA layers.
        """
        n = self.Q.shape[0]  # Number of qubits based on QUBO matrix size
        gammas = params[:self.p]  # Extract gamma parameters for cost Hamiltonian evolution
        betas = params[self.p:]  # Extract beta parameters for mixer Hamiltonian
        # Apply Hadamard gates to all qubits to initialize in uniform superposition
        for i in range(n):
            qml.Hadamard(wires=i)

        # Apply QAOA layers consisting of cost and mixer Hamiltonians
        for layer in range(self.p):
            self.qubo_cost(gammas[layer])  # Apply cost Hamiltonian with corresponding gamma
            self.mixer(betas[layer])  # Apply mixer Hamiltonian with corresponding beta

    def qubo_cost(self, gamma):
        """
        Implements the cost Hamiltonian evolution for the QUBO problem.

        Args:
            gamma (float): Parameter for cost Hamiltonian evolution.
        """
        n = self.Q.shape[0]  # Get number of qubits
        for i in range(n):
            for j in range(n):
                if self.Q[i, j] != 0:  # If the QUBO coefficient is non-zero
                    if i == j:
                        qml.RZ(2 * gamma * float(self.Q[i, j]), wires=i)  # Apply single-qubit phase rotation
                    else:
                        qml.CNOT(wires=[i, j])  # Apply CNOT before controlled rotation
                        qml.RZ(2 * gamma * float(self.Q[i, j]), wires=j)  # Apply controlled phase rotation
                        qml.CNOT(wires=[i, j])  # Undo entanglement with another CNOT

    def mixer(self, beta):
        """
        Implements the mixer Hamiltonian for QAOA.

        Args:
            beta (float): Parameter for mixer Hamiltonian evolution.
        """
        for i in range(self.Q.shape[0]):
            qml.RX(2 * beta, wires=i)  # Apply X-rotation to all qubits

    def prepare_cost_hamiltonian(self):
        """
        Constructs the cost Hamiltonian for the QUBO problem.

        Returns:
            qml.Hamiltonian: The constructed cost Hamiltonian.
        """
        n = self.Q.shape[0]  # Get the size of the QUBO matrix
        coeffs = []  # Store the coefficients of the Hamiltonian terms
        ops = []  # Store the corresponding Pauli operators

        for i in range(n):
            for j in range(n):
                if self.Q[i, j] != 0:  # Only consider non-zero QUBO elements
                    if i == j:
                        coeffs.append(self.Q[i, j])  # Add coefficient for single-qubit term
                        ops.append(qml.PauliZ(i))  # Add Pauli-Z operator
                    else:
                        coeffs.append(self.Q[i, j])  # Add coefficient for two-qubit interaction
                        ops.append(qml.PauliZ(i) @ qml.PauliZ(j))  # Add ZZ interaction term

        return qml.Hamiltonian(coeffs, ops)  # Return the constructed Hamiltonian



class RL_QAOA():
    def __init__(self, Q, n_c,init_paramter,b_vector, QAOA_depth,gamma=0.99,learning_rate_init=0.001):
        self.Q = Q
        self.n_c = n_c
        self.param = init_paramter
        self.b = b_vector
        self.p = QAOA_depth
        self.qaoa_layer = QAOA_layer(QAOA_depth,Q)
        self.gamma = gamma
        self.optimzer = AdamOptimizer([init_paramter,b_vector],learning_rate_init=learning_rate_init)




    def RL_QAOA(self,episodes,epochs,correc_ans = None):
        ## RL_QAOA실행 episodes : 몬테카를로를 진행할 숫자
        ## epochs : parameter 최적화를 진행할 숫자

        for j in range(epochs):
          value_list = []
          QAOA_diff_list = []
          beta_diff_list = []
          if correc_ans is not None:
            prob = 0


          for i in tqdm(range(episodes)):
            QAOA_diff,beta_diff,value = self.rqaoa_execute()
            value_list.append(value)
            QAOA_diff_list.append(QAOA_diff)
            beta_diff_list.append(beta_diff)

            if correc_ans is not None:
              if value <= correc_ans+0.01 and value >= correc_ans-0.01:
                prob+=1


          batch_maen = np.array(value_list) - np.mean(np.array(value_list),axis=0)
          batch_plus = np.where(batch_maen < 0, batch_maen, 0)
          softmaxed_rewards = signed_softmax_rewards(np.array(batch_plus),beta=1)*episodes
          print(batch_plus)

          for index,val in enumerate(softmaxed_rewards):
            QAOA_diff_list[index] *= -val
            beta_diff_list[index] *= -batch_maen[index]


          QAOA_diff_sum = np.mean(QAOA_diff_list,axis=0)
          beta_diff_sum = np.mean(beta_diff_list,axis=0)
          value_sum = np.mean(value_list)
          print(f'QAOA diff : {QAOA_diff_sum}')
          print(f'cost : {value_sum}')
          if correc_ans is not None:
              prob = prob/episodes
              print(f'prob : {prob}')
          update = self.optimzer.get_updates([QAOA_diff_sum,beta_diff_sum])
          print(f'QAOA update : {update[0]}')
          self.param += np.array(update[0])
          self.b += np.reshape(update[1],-1)
          print(f'QAOA param : {self.param}')
          print(f'b param : {self.b}')



    def rqaoa_execute(self,cal_grad = True):
        ## RQAOA를 확률적 뽑기로 진행해 주는 함수
        Q_init = copy.deepcopy(self.Q)
        Q_action = copy.deepcopy(self.Q)
        self.same_list = []
        self.diff_list = []
        self.node_assignments = {}  # Keep track of node assignments
        self.edge_expectations = [] # Edge들의 expectation value 저장(Gradient 계산에 필요)
        self.edge_expectations_grad = [] # Edge들의 expectation value gradient 저장(Gradient 계산에 필요)
        self.policys = [] #각 edge가 지정될 확률을 저장(Gradient 계산에 필요)




        QAOA_diff_list = [] ##QAOA layer parameter gradient
        beta_diff_list = [] ##beta parameter gradient
        index = 0
        while Q_init.shape[0] > self.n_c:
            # Select and cut an edge,cal grad
            edge_expectations = self._qaoa_edge_expectations(Q_init,[i for i in range(self.p*index*2,self.p*index*2+2*self.p)])
            selected_edge_idx, policy, edge_res = self._select_edge_to_cut(Q_action,Q_init,edge_expectations)
            if cal_grad:
              #edge_res_grad = self._qaoa_edge_expectations_gradients(Q_init, [i for i in range(self.p*index*2,self.p*index*2+2*self.p)])
              edge_res_grad = self._qaoa_edge_expectations_gradient(Q_init, [i for i in range(self.p*index*2,self.p*index*2+2*self.p)],selected_edge_idx)
            if cal_grad:
              #QAOA_diff = self._compute_log_pol_diff(selected_edge_idx,Q_action,edge_res,edge_res_grad ,policy)
              QAOA_diff = self._compute_log_pol_diff_idx(selected_edge_idx,Q_action,edge_res,edge_res_grad ,policy)*self.gamma**(Q_init.shape[0]-index)
              beta_diff = self._compute_grad_beta(selected_edge_idx,Q_action,policy,edge_res)*self.gamma**(Q_init.shape[0]-index)


              QAOA_diff_list.append(QAOA_diff)
              beta_diff_list.append(beta_diff)
            Q_init,Q_action = self._cut_edge(selected_edge_idx,edge_res,Q_action,Q_init)
            index+=1


        self.QAOA_result = copy.deepcopy(self.node_assignments)
        # 그래프의 크기가 n_c보다 작으면 전체 space를 다 계산해서 최종값 도출
        self._brute_force_optimal(Q_init)


        Value = self._state_energy(np.array(self.node_assignments),self.Q)

        ## 최근에 얻은 값이 최종 변화율에 영향을 크게 미치도록 함수 변경(큰 그래프에서는 뭘 선택하든 중요하지 않을때가 많음)


        ## 정책에 대한 미분과 panalty 받은 보상을 곱해서 최종 기울기 도출
        if cal_grad:

          beta_diff_res = None
          for beta_diff in beta_diff_list:
            if beta_diff_res is None:
              beta_diff_res = beta_diff
            else:
              beta_diff_res += beta_diff

          QAOA_diff_res = None
          for QAOA_diff in QAOA_diff_list:
            if QAOA_diff_res is None:
              QAOA_diff_res = QAOA_diff
            else:
              QAOA_diff_res += QAOA_diff

          return QAOA_diff_res,beta_diff_res,Value


        else:
          return Value




    def _compute_log_pol_diff(self, idx,Q_action,edge_expectations,edge_expectations_grad,policy):
        ## 특정 edge가 선택 됐을 때 그 정책에 대한 paramter의 log기울기 값 계산
        action_space = self._action_space(Q_action)
        betas = self.b[action_space]
        gather = np.zeros_like(policy)
        for i in range(len(edge_expectations_grad)):
            gather[i] += policy[i] * betas[i]

        diff_log_pol = betas[idx] *np.sign(edge_expectations[idx])* edge_expectations_grad[idx]
        for i in range(len(gather)):
            if gather[i]:
                diff_log_pol-= gather[i] * np.sign(edge_expectations[i]) * (
                    (edge_expectations_grad[i]))

        return np.array(diff_log_pol)

    def _compute_log_pol_diff_idx(self, idx,Q_action,edge_expectations,grad,policy):
        ## 특정 edge가 선택 됐을 때 그 정책에 대한 paramter의 log기울기 값 계산
        action_space = self._action_space(Q_action)
        betas = self.b[action_space]
        diff_log_pol = betas[idx] *np.sign(edge_expectations[idx])* grad - policy[idx] * betas[idx] *np.sign(edge_expectations[idx])* grad
        return np.array(diff_log_pol)

    def _compute_grad_beta(self,idx,Q_action,policy,edge_expectations):

        grad_beta = []
        abs_expectations = abs(np.array(edge_expectations))
        action_space = self._action_space(Q_action)

        betas_idx = action_space

        grad = np.zeros(len(self.b))
        grad[betas_idx[idx]] += abs_expectations[idx]
        for i in range(len(action_space)):
            grad[betas_idx[i]] -= policy[i] * abs_expectations[i]


        return np.array(grad)



    def _cut_edge(self, selected_edge_idx,expectations,Q_action,Q_init):
        ## 선택된 edge에 해당하는 index와 그에 해당하는 expectaion value를 받아 1 혹은 -1 를 edge의 노드에 부여하는 메소드
        ## 또한 Q_action,줄어들어진 Q도 여기서 갱신해준다
        edge_list = [(i, j) for i in range(Q_init.shape[0]) for j in range(Q_init.shape[0]) if Q_init[i, j] != 0 and i!=j]
        edge_to_cut = edge_list[selected_edge_idx]
        edge_to_cut = sorted(edge_to_cut)

        expectation = expectations[selected_edge_idx]
        ###수정


        i = edge_to_cut[0]
        j = edge_to_cut[1]

        for key in dict(sorted(self.node_assignments.items(), key=lambda item: item[0])):
          if i >= key:
            i+=1
          if j >= key:
            j+=1

        self.node_assignments[i] = 1

        new_Q,Q_action = reduce_hamiltonian(Q_init,edge_to_cut[0],edge_to_cut[1],self.node_assignments,int(np.sign(expectation)))
        if expectation > 0:
          self.same_list.append((i,j))
        else:
          self.diff_list.append((i,j))


        return new_Q,Q_action

    def _select_edge_to_cut(self,Q_action,Q_init,edge_expectations):
          ## ZZ를 측정하여 beta와 곱한 후 softmax를 진행하여 자를 edge를 확률적으로 고르는 method
          action_space = self._action_space(Q_action)
          try:
            interactions = abs(np.array((edge_expectations))) * self.b[action_space]
          except:
            print(abs(np.array((edge_expectations))),self.b[action_space])
            raise ValueError("Invalid input",action_space,abs(np.array((edge_expectations))))
          probabilities = torch.softmax(torch.tensor(interactions), dim=0).numpy()
          selected_edge_idx = np.random.choice(len(probabilities), p=probabilities)
          return selected_edge_idx, probabilities, edge_expectations




    def _action_space(self,Q_action):
        ## 현재 살아있는 edge들이 기존 그래프에서 몇번째 edge였는지 표시해주는 action space뽑아내주는 함수
        ## Q_action : 죽은 노드들이 있는 행과 열에 대하여 0으로 대체된 살아있는 노드들만 표시되는 매트릭스
        action_space_list = []
        index = 0
        for i in range(Q_action.shape[0]):
            for j in range(Q_action.shape[0]):
                if i!=j:
                    if Q_action[i,j] != 0:
                        action_space_list.append(index)
                    index+=1
        return action_space_list

    def _qaoa_edge_expectations(self, Q,idx):
        ## 각 edge별 ZZ interaction을 도출해주는 함수
        self.qaoa_layer = QAOA_layer(self.p,Q)
        @qml.qnode(self.qaoa_layer.dev)
        def circuit(param):
            self.qaoa_layer.qaoa_circuit(param)
            return [qml.expval(qml.PauliZ(i) @ qml.PauliZ(j)) for i in range(Q.shape[0]) for j in range(Q.shape[0]) if Q[i, j] != 0 and i!=j]

        return circuit(self.param[idx])

    def _qaoa_edge_expectations_gradients(self, Q,idx):
      ## 각 edge별 ZZ interaction의 기울기를 도출해주는 함수
        self.qaoa_layer = QAOA_layer(self.p,Q)
        res = []
        @qml.qnode(self.qaoa_layer.dev)
        def circuit(params,i,j):
            self.qaoa_layer.qaoa_circuit(params[idx])
            return qml.expval(qml.PauliZ(i) @ qml.PauliZ(j))
        for i in range(Q.shape[0]) :
          for j in range(Q.shape[0]):
             if Q[i, j] != 0 and i != j:
                # Compute gradients for each parameter
                gradients = qml.grad(circuit)(self.param,i,j)
                res.append(gradients)
        return res


    def _qaoa_edge_expectations_gradient(self, Q,idx,index):
      ## 각 edge별 ZZ interaction의 기울기를 도출해주는 함수
      self.qaoa_layer = QAOA_layer(self.p,Q)
      res = []
      number = 0
      @qml.qnode(self.qaoa_layer.dev)
      def circuit(params,i,j):
          self.qaoa_layer.qaoa_circuit(params[idx])
          return qml.expval(qml.PauliZ(i) @ qml.PauliZ(j))
      for i in range(Q.shape[0]) :
        for j in range(Q.shape[0]):
            if Q[i, j] != 0 and i != j:
              if number == index:
                # Compute gradients for each parameter
                gradients = qml.grad(circuit)(self.param,i,j)
                return gradients
              number+=1




    def _brute_force_optimal(self,Q):


        n = self.Q.shape[0]
        configs = list(itertools.product([-1, 1], repeat=n))
        best_value = np.inf
        best_config = None
        res_node = None
        comb_list = find_valid_combinations(n,self.same_list,self.diff_list)
        #comb_list = get_case(self.same_list,self.diff_list,n)
        ''' if len(comb_list) != len(comb_list_fast):
          print('something happened ... ' ,comb_list_fast,comb_list)
          raise '''
        for comb in comb_list:
          value = self._state_energy(np.array(comb),self.Q)
          if value < best_value:
              best_value = value
              res_node = copy.copy(comb)

        self.node_assignments = res_node





    def _state_energy(self,state,Q):
        ## State가 주어질 때 그 에너지를 계산해주는 함수


        # 동일한 크기의 단위 행렬 생성
        identity_matrix = np.eye(Q.shape[0], dtype=bool)

        # 대각선 요소를 제거한 새로운 행렬 생성
        interaction = np.where(identity_matrix, 0, Q)
        diagonal_elements = np.diag(Q)
        value = diagonal_elements@state + state.T@interaction@state
        return value

    def _dict_to_list(self,node_assign):
        ##보조 함수 assign된 node들의 0 과 1을 list 형태로 순서대로 출력해주는 함수
        node_assignments_list = [0]*len(node_assign)
        for key,value in node_assign.items(): # Change: Iterate through key-value pairs using .items()
            node_assignments_list[key]=value
        return np.array(node_assignments_list)


''' # RL-QAOA setup
init_params = np.random.uniform(0, np.pi, 2 * depth*5)
rl_qaoa = RL_QAOA(Q,Q.shape[0],init_params,b_vector = np.array([10.]*int((Q.shape[0]**2))) ,QAOA_depth=depth,gamma = 0.99,learning_rate_init=[0.02,0.00])
final_config = rl_qaoa.rqaoa_execute()
#rl_qaoa.n_c = 2
print(f"classical_result : {final_config},best : {rl_qaoa.node_assignments}" )
rl_qaoa.n_c = 2
#rl_qaoa.rqaoa_execute()
# Execute RQAOA
final_config = rl_qaoa.RL_QAOA(episodes=150,epochs=60,correc_ans=float(final_config[2])) '''

' # RL-QAOA setup\ninit_params = np.random.uniform(0, np.pi, 2 * depth*5)\nrl_qaoa = RL_QAOA(Q,Q.shape[0],init_params,b_vector = np.array([10.]*int((Q.shape[0]**2))) ,QAOA_depth=depth,gamma = 0.99,learning_rate_init=[0.02,0.00])\nfinal_config = rl_qaoa.rqaoa_execute()\n#rl_qaoa.n_c = 2\nprint(f"classical_result : {final_config},best : {rl_qaoa.node_assignments}" )\nrl_qaoa.n_c = 2\n#rl_qaoa.rqaoa_execute()\n# Execute RQAOA\nfinal_config = rl_qaoa.RL_QAOA(episodes=150,epochs=60,correc_ans=float(final_config[2])) '

In [432]:
def generate_upper_triangular_qubo(size, low=-10, high=10, integer=True, seed=None):
    """
    Generates an upper-triangular QUBO (Quadratic Unconstrained Binary Optimization) matrix.

    Args:
        size (int): The number of variables (size of the QUBO matrix).
        low (int/float): Minimum value of the random elements.
        high (int/float): Maximum value of the random elements.
        integer (bool): If True, generates integer values; otherwise, generates float values.
        seed (int, optional): Random seed for reproducibility.

    Returns:
        np.ndarray: An upper-triangular QUBO matrix of the specified size.
    """
    if seed is not None:
        np.random.seed(seed)

    # Generate random values for the upper triangular part including diagonal
    if integer:
        Q = np.random.randint(low, high, (size, size))
    else:
        Q = np.random.uniform(low, high, (size, size))

    # Keep only the upper triangle values (including diagonal), set lower triangle to zero
    Q = np.triu(Q)


    # Ensure diagonal values are positive (bias terms)
    np.fill_diagonal(Q,np.diagonal(Q))


    return Q

# Example usage
qubo_matrix = generate_upper_triangular_qubo(size=3, low=-5, high=5, integer=True, seed=70)
print("Randomly generated upper-triangular QUBO matrix:\n", qubo_matrix)

import numpy as np

# Original QUBO matrix
qubo_matrix = np.array([
    [0.0658492, -0.024853287,  0.064008761,  0.058160105,  0.058757735],
    [-0.024853287, 0.080128677, -0.025012106, -0.023698929, -0.013849719],
    [0.064008761, -0.025012106,  0.063201845,  0.053540045,  0.055350148],
    [0.058160105, -0.023698929,  0.053540045,  0.066177288,  0.055601435],
    [0.058757735, -0.013849719,  0.055350148,  0.055601435,  0.059764565]
])

diagonal = np.diag(np.diag(qubo_matrix)) + np.eye(qubo_matrix.shape[0])

# Combine results: upper triangle + modified diagonal






# Create an upper triangular matrix by doubling the upper triangle values and keeping diagonal
modified_qubo = np.triu(qubo_matrix, k=1) * 2 + np.diag(np.diag(qubo_matrix))
modified_qubo =  modified_qubo + diagonal

print("Modified QUBO Matrix:\n", modified_qubo)
qubo_matrix = modified_qubo


Randomly generated upper-triangular QUBO matrix:
 [[ 1 -3  3]
 [ 0 -1  0]
 [ 0  0 -1]]
Modified QUBO Matrix:
 [[ 1.1316984  -0.04970657  0.12801752  0.11632021  0.11751547]
 [ 0.          1.16025735 -0.05002421 -0.04739786 -0.02769944]
 [ 0.          0.          1.12640369  0.10708009  0.1107003 ]
 [ 0.          0.          0.          1.13235458  0.11120287]
 [ 0.          0.          0.          0.          1.11952913]]


In [433]:
import numpy as np
from itertools import combinations
import random
def cut_k_matrices_randomly(matrix, m, k):
    n = matrix.shape[0]
    if m >= n:
        raise ValueError("m must be smaller than the size of the matrix")

    selected_subsets = []
    remaining_indices = list(range(n))
    combinations_list = generate_combinations(n, m)
    random_indices = random.sample(range(len(combinations_list)), k)
    indices = [combinations_list[i] for i in random_indices]
    for indice in indices:
        matrix_copy = copy.deepcopy(matrix)
        remaining_indices = list(range(n))
        remaining_indices = sorted(set(remaining_indices) - set(indice))
        reduced_matrix = matrix_copy[np.ix_(remaining_indices, remaining_indices)]
        selected_subsets.append(reduced_matrix)

    return selected_subsets

def generate_combinations(n, m):
    return list(combinations(range(n), m))




In [434]:
class QAOA_pretrain():
  def __init__(self,Q,size,depth,number):
    self.Q = Q
    self.size = size
    self.p = depth
    self.number = number


  def qaoa_exp(self,param):
    Q_action = copy.deepcopy(self.Q)
    Q_list = cut_k_matrices_randomly(Q_action, self.size,self.number)
    res = 0
    for Q in Q_list:
      qaoa = QAOA_layer(self.p, Q)
      @qml.qnode(qaoa.dev)
      def qaoa_expectation(param):
          """
          Quantum node that runs the QAOA circuit and measures the expectation value
          of the cost Hamiltonian.

          Args:
              params (np.ndarray): Array of QAOA parameters (gammas and betas).

          Returns:
              float: Expectation value of the cost Hamiltonian.
          """
          qaoa.qaoa_circuit(param)
          return qml.expval(qaoa.ham)
      res += qaoa_expectation(param)
    return res/len(Q_list)
  # Define the cost function for optimization
  def cost_function(self,params):
      """
      Computes the expectation value of the cost Hamiltonian.

      Args:
          params (np.ndarray): Array of QAOA parameters (gammas and betas).

      Returns:
          float: Expectation value of the cost Hamiltonian.
      """
      return self.qaoa_exp(params)


In [435]:
import pennylane as qml
from pennylane import numpy as np
from scipy.optimize import minimize

# Define QAOA depth

''' Q = qubo_matrix
print(Q)
depth = 2 '''


depth = 1
size = 9
Q = generate_upper_triangular_qubo(size,-10,10,integer=True)
print(Q)
param_list = []

start = 0
while size>2:
    if len(generate_combinations(Q.shape[0], start))>5:
        reps = 5
    else:
        reps = len(generate_combinations(Q.shape[0], start))
    QAOA_pre = QAOA_pretrain(Q,start,depth,reps)
    init_params = np.random.uniform(0, np.pi, 2 * depth)


    # Optimize the QAOA parameters using classical optimizer (COBYLA)
    result = minimize(QAOA_pre.cost_function, init_params, method="COBYLA", options={'maxiter': 200})

    # Extract optimized parameters and minimum cost
    param = result.x
    min_cost = result.fun
    param_list.append(param)
    # Print the results
    print("Optimized parameters (gammas and betas):", param)
    print("Minimum cost achieved:", min_cost)
    size = size-1
    start +=1

[[ -3  -2 -10  -9   8  -9   6   5  -1]
 [  0   5  -8   5   0   0   1  -7   4]
 [  0   0  -1  -8   9   3   7  -4   6]
 [  0   0   0   5   3   5  -5  -4   7]
 [  0   0   0   0  -4   9   2  -6  -6]
 [  0   0   0   0   0   3  -3  -6   2]
 [  0   0   0   0   0   0   8   4  -4]
 [  0   0   0   0   0   0   0  -1   8]
 [  0   0   0   0   0   0   0   0   7]]
Optimized parameters (gammas and betas): [1.59726677 3.54191737]
Minimum cost achieved: -7.910499770474819
Optimized parameters (gammas and betas): [3.75843796 1.95994033]
Minimum cost achieved: 1.0429469520510386
Optimized parameters (gammas and betas): [-0.03641158  3.39384995]
Minimum cost achieved: -31.27610007835504
Optimized parameters (gammas and betas): [1.54336154 3.29777235]
Minimum cost achieved: -3.3253365314132663
Optimized parameters (gammas and betas): [0.46688391 2.70648995]
Minimum cost achieved: -2.7588047886671463
Optimized parameters (gammas and betas): [2.07343724 1.88774116]
Minimum cost achieved: 0.4307986927285009
Op

In [436]:
import pennylane as qml
from pennylane import numpy as np

# Define the QUBO matrix for the problem
def generate_upper_triangular_qubo(size, low=-10, high=10, integer=True, seed=None):
    """
    Generates an upper-triangular QUBO (Quadratic Unconstrained Binary Optimization) matrix.

    Args:
        size (int): The number of variables (size of the QUBO matrix).
        low (int/float): Minimum value of the random elements.
        high (int/float): Maximum value of the random elements.
        integer (bool): If True, generates integer values; otherwise, generates float values.
        seed (int, optional): Random seed for reproducibility.

    Returns:
        np.ndarray: An upper-triangular QUBO matrix of the specified size.
    """
    if seed is not None:
        np.random.seed(seed)

    # Generate random values for the upper triangular part including diagonal
    if integer:
        Q = np.random.randint(low, high, (size, size))
    else:
        Q = np.random.uniform(low, high, (size, size))

    # Keep only the upper triangle values (including diagonal), set lower triangle to zero
    Q = np.triu(Q)

    # Ensure diagonal values are positive (bias terms)
    np.fill_diagonal(Q, np.abs(np.diagonal(Q)))

    return Q



# Initialize the QAOA layer
qaoa = QAOA_layer(depth, Q)

# Define the QNode for optimization
@qml.qnode(qaoa.dev)
def qaoa_expectation(params):
    """
    Quantum node that runs the QAOA circuit and measures the expectation value
    of the cost Hamiltonian.

    Args:
        params (np.ndarray): Array of QAOA parameters (gammas and betas).

    Returns:
        float: Expectation value of the cost Hamiltonian.
    """
    qaoa.qaoa_circuit(params)
    return qml.expval(qaoa.ham)

# Define the cost function for optimization
def cost_function(params):
    """
    Computes the expectation value of the cost Hamiltonian.

    Args:
        params (np.ndarray): Array of QAOA parameters (gammas and betas).

    Returns:
        float: Expectation value of the cost Hamiltonian.
    """
    return qaoa_expectation(params)





'''

opt = qml.AdamOptimizer(stepsize=0.1)

# Initial random parameters for QAOA (gammas and betas)
init_params = optimized_params

# Optimization loop
max_iterations = 200
params = init_params
for i in range(max_iterations):
    params = opt.step(cost_function, params)
    if i % 20 == 0:
        cost = cost_function(params)
        print(f"Iteration {i}: Cost = {cost}")

# Final optimized parameters
final_cost = cost_function(params)
print("Optimized parameters (gammas and betas):", params)
print("Final minimum cost achieved:", final_cost) '''





'\n\nopt = qml.AdamOptimizer(stepsize=0.1)\n\n# Initial random parameters for QAOA (gammas and betas)\ninit_params = optimized_params\n\n# Optimization loop\nmax_iterations = 200\nparams = init_params\nfor i in range(max_iterations):\n    params = opt.step(cost_function, params)\n    if i % 20 == 0:\n        cost = cost_function(params)\n        print(f"Iteration {i}: Cost = {cost}")\n\n# Final optimized parameters\nfinal_cost = cost_function(params)\nprint("Optimized parameters (gammas and betas):", params)\nprint("Final minimum cost achieved:", final_cost) '

In [437]:
params = param_list[0]
dev = qml.device("default.qubit", shots=1500)
@qml.qnode(dev)
def sample_solution(params):
    """Runs the optimized QAOA circuit and samples the output state."""
    qaoa.qaoa_circuit(params)
    return qml.sample()

# Get multiple samples from the optimized circuit
num_samples = 1000
samples = sample_solution(params)

# Convert measurement results from {-1,1} to {0,1}
binary_solutions = samples*(-2)+1

# Compute the cost for each sampled solution
def evaluate_solution(sample):
    # 동일한 크기의 단위 행렬 생성
    identity_matrix = np.eye(Q.shape[0], dtype=bool)

    # 대각선 요소를 제거한 새로운 행렬 생성
    interaction = np.where(identity_matrix, 0, Q)
    diagonal_elements = np.diag(Q)
    value = diagonal_elements@sample + sample.T@interaction@sample
    return value

# Evaluate the cost for all sampled solutions and find the best
costs = np.array([evaluate_solution(sample) for sample in binary_solutions])
best_solution = binary_solutions[np.argmin(costs)]
best_cost = np.min(costs)

print("Best solution found:", best_solution)
print("Cost of best solution:", best_cost)

Best solution found: [ 1  1  1  1 -1  1 -1  1 -1]
Cost of best solution: -119


In [438]:
dev = qml.device("default.qubit", shots=1500)
@qml.qnode(dev)
def sample_solution(params):
    """Runs the optimized QAOA circuit and samples the output state."""
    qaoa.qaoa_circuit(params)
    return qml.sample()

In [439]:
# Initial parameters for QAOA
init_params = params
init_params = np.reshape(init_params,(-1))

# RL-QAOA setup
rl_qaoa = RL_QAOA(Q,Q.shape[0],init_params,b_vector = np.array([25.]*30) ,QAOA_depth=1,gamma = 0.99,learning_rate_init=[0.01,0.05])
final_config = rl_qaoa.rqaoa_execute()
rl_qaoa.n_c = 1
print(f"classical_result : {final_config},best : {rl_qaoa.node_assignments}" )

classical_result : (None, None, tensor(-119, requires_grad=True)),best : [1, 1, 1, 1, -1, 1, -1, 1, -1]


In [440]:
prob = 0
for cost in costs:
  if cost == float(final_config[2]):
    prob +=1

prob = prob/len(costs)
print('success probobility : ',prob)

success probobility :  0.0026666666666666666


In [441]:
### Single optimization
# Example QUBO matrix
#Q = Q_onsite

# Initial parameters for QAOA
init_params = np.array(param_list)


#init_params = np.random.uniform(0, np.pi, 2 * depth*20)
init_params = np.reshape(init_params,(-1))

# RL-QAOA setup
rl_qaoa = RL_QAOA(Q,Q.shape[0],init_params,b_vector = np.array([20.]*int((Q.shape[0]**2))) ,QAOA_depth=depth,gamma = 0.97,learning_rate_init=[0.02,0.05])
final_config = rl_qaoa.rqaoa_execute()
rl_qaoa.n_c = 3
print(f"classical_result : {final_config},best : {rl_qaoa.node_assignments}" )


# Execute RQAOA
final_config = rl_qaoa.RL_QAOA(episodes=50,epochs=60,correc_ans=float(final_config[2]))

classical_result : (None, None, tensor(-119, requires_grad=True)),best : [1, 1, 1, 1, -1, 1, -1, 1, -1]


100%|██████████| 50/50 [00:37<00:00,  1.34it/s]


[-23.48  -1.48   0.    -1.48  -5.48   0.     0.     0.     0.     0.
   0.     0.   -25.48  -1.48 -23.48 -23.48 -25.48   0.    -5.48 -25.48
 -25.48  -1.48   0.   -25.48 -47.48 -23.48  -5.48   0.     0.     0.
  -1.48   0.    -5.48   0.     0.   -13.48 -23.48   0.   -25.48   0.
 -47.48 -23.48 -23.48  -7.48   0.    -5.48   0.   -25.48   0.     0.  ]
QAOA diff : [ 69.98358488   2.53868216 -88.573714     1.12521403  19.46375032
   5.22452983  23.59400002  16.96461418  63.43277814  -0.11016577
  -7.51449612   1.23408342   0.           0.        ]
cost : -71.52
prob : 0.04
QAOA update : [ 0.02        0.02       -0.02        0.01999999  0.02        0.02
  0.02        0.02        0.02       -0.01999994 -0.02        0.01999999
  0.          0.        ]
QAOA param : [ 1.61726677  3.56191736  3.73843796  1.97994032 -0.01641158  3.41384995
  1.56336154  3.31777235  0.48688391  2.68649     2.05343724  1.90774115
  3.20109591 -0.31554568]
b param : [19.95000426 20.04999815 20.04999996 19.95000002 20

100%|██████████| 50/50 [00:36<00:00,  1.37it/s]


[ -9.52   0.   -19.52   0.   -29.52 -29.52   0.     0.     0.   -21.52
  -1.52 -29.52 -19.52   0.   -29.52 -19.52 -29.52 -43.52   0.     0.
  -9.52   0.   -19.52   0.     0.     0.    -1.52   0.     0.     0.
  -1.52 -19.52 -43.52   0.   -43.52   0.     0.    -1.52 -19.52 -43.52
   0.   -43.52  -9.52 -19.52   0.    -9.52 -29.52 -43.52  -1.52 -29.52]
QAOA diff : [-105.86975642    1.59961991    9.67745664   -0.24139489 -154.40367997
    7.55913739 -135.36260589    5.09787376    5.46066893   -1.14709921
   -0.73870277    3.99279249    0.            0.        ]
cost : -75.48
prob : 0.12
QAOA update : [-0.00502984  0.01927328 -0.01170456  0.00997945 -0.01309077  0.01986195
 -0.01236186  0.01711872  0.01462884 -0.01609514 -0.01479355  0.01817516
  0.          0.        ]
QAOA param : [ 1.61223693  3.58119065  3.7267334   1.98991977 -0.02950235  3.4337119
  1.55099968  3.33489107  0.50151275  2.67039487  2.03864369  1.92591632
  3.20109591 -0.31554568]
b param : [19.91121077 20.08891467 20.09

...

[-17.68 -17.68 -17.68 -17.68  -3.68   0.   -17.68   0.   -17.68   0.
   0.     0.   -17.68 -17.68 -17.68 -17.68   0.   -17.68 -17.68 -17.68
   0.     0.   -17.68 -17.68   0.   -17.68 -17.68 -17.68 -17.68   0.
 -17.68 -17.68   0.   -17.68 -17.68   0.   -17.68   0.   -17.68 -17.68
 -17.68 -17.68 -17.68 -17.68 -17.68   0.   -17.68 -17.68 -17.68   0.  ]
QAOA diff : [-3.56759586  0.25645504 -3.0408422   0.01665476 -1.91635669  0.25313071
  4.18171181  0.03869325 -0.98933834  0.0857436   0.77663248  0.10166303
  0.          0.        ]
cost : -101.32
prob : 0.68
QAOA update : [-0.00046449  0.00146453  0.00015109 -0.00069985 -0.00044389  0.00092146
  0.00019743 -0.00136408 -0.00126053 -0.00115205 -0.00082661  0.00135696
  0.          0.        ]
QAOA param : [ 1.60344287  3.58877145  3.7327047   2.01257383 -0.03250509  3.60773943
  1.54512193  3.6299513   0.65888751  2.34372411  1.9145922   2.27863328
  3.20109591 -0.31554568]
b param : [19.44180336 19.64442978 22.36061382 17.86934056 19.4819

100%|██████████| 50/50 [00:36<00:00,  1.37it/s]


[  0.     0.   -22.28   0.     0.   -22.28   0.   -22.28 -22.28   0.
  -0.28 -22.28 -22.28   0.   -22.28 -22.28   0.   -22.28   0.   -22.28
   0.     0.   -22.28 -22.28 -22.28 -22.28   0.   -22.28 -22.28 -22.28
   0.     0.   -22.28 -22.28 -22.28 -22.28 -22.28 -22.28 -22.28 -22.28
   0.     0.     0.     0.   -22.28  -8.28 -22.28 -22.28   0.   -22.28]
QAOA diff : [ 1.97403867 -0.00877936 -1.19332639  0.02674684 -5.63626013  0.09057662
  2.54416633 -0.19707001  7.79842694 -0.12410232 -0.28532256  0.15664399
  0.          0.        ]
cost : -96.72
prob : 0.58
QAOA update : [-3.18108911e-04  1.31007687e-03  3.82320350e-05 -5.81367253e-04
 -5.99379825e-04  9.10401684e-04  3.04586895e-04 -1.33576852e-03
 -6.56405614e-04 -1.15052148e-03 -7.64510086e-04  1.33686269e-03
  0.00000000e+00  0.00000000e+00]
QAOA param : [ 1.60312477  3.59008152  3.73274293  2.01199247 -0.03310447  3.60864983
  1.54542651  3.62861554  0.6582311   2.34257359  1.91382769  2.27997014
  3.20109591 -0.31554568]
b param 

100%|██████████| 50/50 [00:36<00:00,  1.37it/s]

[  0.     0.     0.   -20.92 -20.92   0.     0.     0.   -20.92 -20.92
   0.   -20.92   0.   -20.92 -20.92   0.   -20.92 -20.92 -20.92 -20.92
 -20.92   0.   -20.92 -20.92 -20.92 -20.92   0.   -20.92 -20.92   0.
   0.   -20.92 -20.92 -20.92 -20.92 -20.92 -20.92 -20.92   0.   -20.92
   0.   -20.92 -20.92   0.   -20.92 -20.92 -20.92 -20.92   0.     0.  ]
QAOA diff : [ 1.93973407  0.05924651 -1.7493334   0.02846404  1.02997306  0.10502234
  0.92767532  0.06753166 -0.43175216  0.09940566  1.08787455  0.10754553
  0.          0.        ]
cost : -98.08
prob : 0.64
QAOA update : [-0.00018614  0.00132075 -0.00011152 -0.00046981 -0.00050781  0.00091278
  0.00032256 -0.00117869 -0.00062299 -0.00095961 -0.00063947  0.00128615
  0.          0.        ]
QAOA param : [ 1.60293863  3.59140228  3.73263141  2.01152266 -0.03361228  3.60956261
  1.54574907  3.62743685  0.65760812  2.34161398  1.91318822  2.2812563
  3.20109591 -0.31554568]
b param : [19.44300248 19.633499   22.46729937 17.77748507 19.4586

In [338]:
from pennylane import numpy as np
from scipy.optimize import minimize


# Initial parameters for QAOA
init_params = np.array(param_list)


#init_params = np.random.uniform(0, np.pi, 2 * depth*5)
init_params = np.reshape(init_params,(-1))

# RL-QAOA setup
rl_qaoa = RL_QAOA(Q,2,init_params,b_vector = np.array([25.]*70) ,QAOA_depth=depth,gamma = 0.99,learning_rate_init=[0.02,0.000])
# 최적화할 목적 함수 정의 (x만 최적화하고, 일부 값은 고정)
def objective_function(params):
    rl_qaoa.param = np.array(params[:len(rl_qaoa.param)])
    rl_qaoa.b = np.array(params[len(rl_qaoa.param):])
    result = 0
    for i in range(150):
      res = rl_qaoa.rqaoa_execute(False)
      result += float(res)
    result/=150
    return result

# 두 번째와 세 번째 인수 고정
fixed_val1 = 3.0
fixed_val2 = 2.0

# 진행 상태 출력을 위한 콜백 함수
iteration_count = 0

def callback(xk):
    global iteration_count
    iteration_count += 1
    if iteration_count % 3 == 0:
        print(f"Iteration {iteration_count}: x = {xk}, objective = {objective_function(xk)}")

# 초기 추정값 (x만 최적화)
x0 = np.array(list(rl_qaoa.param)+list(rl_qaoa.b))
x0 = np.reshape(x0,-1)

# 최적화 수행 (COBYLA 사용)
result = minimize(
    objective_function,  # 고정된 인수 적용
    x0,
    method='COBYLA',
    callback=callback,
    options={'maxiter': 16}  # 최대 100번 반복
)

# 최적화 결과 출력
print("\n최적화 완료!")
print("최적화된 x:", result.x)
print("최적화된 함수 값:", result.fun)
print("성공 여부:", result.success)
print("메시지:", result.message)


Iteration 3: x = [ 0.88364013  2.30475126  3.49863265  1.29203627  2.91238991  0.36583027
  0.7059254   2.23078364  2.88159989  1.96389428  1.01848853  0.71305778
  1.45651194  1.7656242  25.         25.         25.         25.
 25.         25.         25.         25.         25.         25.
 25.         25.         25.         25.         25.         25.
 25.         25.         25.         25.         25.         25.
 25.         25.         25.         25.         25.         25.
 25.         25.         25.         25.         25.         25.
 25.         25.         25.         25.         25.         25.
 25.         25.         25.         25.         25.         25.
 25.         25.         25.         25.         25.         25.
 25.         25.         25.         25.         25.         25.
 25.         25.         25.         25.         25.         25.
 25.         25.         25.         25.         25.         25.        ], objective = -50.17333333333333
Iteration 6: x =

KeyboardInterrupt: 

In [341]:
rl_qaoa.b

tensor([25., 25., 25., 25., 25., 25., 25., 25., 25., 25., 25., 25., 25.,
        25., 25., 25., 25., 25., 25., 25., 25., 25., 25., 25., 25., 25.,
        25., 25., 25., 25., 25., 25., 25., 25., 25., 25., 25., 25., 25.,
        25., 25., 25., 25., 25., 25., 25., 25., 25., 25., 25., 25., 25.,
        25., 25., 25., 25., 25., 25., 25., 25., 25., 25., 25., 25., 25.,
        25., 25., 25., 25., 25.], requires_grad=True)

In [339]:
rl_qaoa.param

tensor([0.88364013, 2.30475126, 3.49863265, 2.29203627, 2.91238991,
        0.36583027, 0.7059254 , 2.23078364, 3.88159989, 1.96389428,
        1.01848853, 0.71305778, 1.45651194, 1.7656242 ], requires_grad=True)

In [ ]:
# RL-QAOA setup
#rl_qaoa = RL_QAOA(Q,Q.shape[0],init_params,b_vector = np.array([25.]*70) ,QAOA_depth=depth,gamma = 0.99,learning_rate_init=[0.02,0.000])
#final_config = rl_qaoa.rqaoa_execute()
#rl_qaoa.n_c = 2
#print(f"classical_result : {final_config},best : {rl_qaoa.node_assignments}" )


# Execute RQAOA
final_config = rl_qaoa.RL_QAOA(episodes=150,epochs=60,correc_ans=float(final_config[2]))

100%|██████████| 150/150 [00:36<00:00,  4.07it/s]


[-16. -16.   0. -16. -16. -16.   0. -16.   0. -16.   0. -20.   0. -16.
   0. -16. -16. -16. -16. -16. -16.   0. -16.   0. -16.   0. -16.   0.
   0.   0. -16. -16. -16. -16. -16. -16. -16. -16. -16.   0. -16.   0.
 -16.   0. -16.   0. -16.   0.   0. -16.   0. -16.   0. -16. -16.   0.
 -16. -16. -16.   0. -16. -16. -16. -16. -16. -16.   0. -16. -16. -16.
 -16. -16. -16. -16. -16. -16. -16.   0. -16. -16.   0. -16.   0. -16.
 -16.   0. -16. -16. -16.   0. -16. -16. -16. -16. -16. -16.   0.   0.
   0. -16. -16. -16.   0.   0.   0. -16. -16.   0.   0. -16. -16. -16.
 -16. -16.   0. -16.   0. -16. -16. -16. -16. -16. -16. -16.   0. -16.
   0. -16.   0. -16.   0.   0. -16.   0. -16.   0. -16. -16. -16. -16.
 -16. -16.   0.   0.   0. -16.   0. -16.   0. -16.]
QAOA diff : [256.09063851 -36.19425484  65.8344376   17.98613328  -3.00731666
  -1.42662768]
cost : -53.0
prob : 0.0
QAOA update : [ 0.02 -0.02  0.02  0.02 -0.02 -0.02]
QAOA param : [ 2.03768174  2.39262379  4.2586235  -1.00456235  2.9341

100%|██████████| 150/150 [00:35<00:00,  4.26it/s]


[-21.98666667   0.           0.         -21.98666667   0.
 -21.98666667 -21.98666667   0.         -21.98666667   0.
 -21.98666667 -21.98666667 -21.98666667 -21.98666667   0.
   0.         -21.98666667   0.         -21.98666667 -21.98666667
 -21.98666667   0.         -21.98666667 -21.98666667 -21.98666667
   0.         -21.98666667   0.           0.           0.
 -21.98666667   0.         -21.98666667 -21.98666667 -21.98666667
 -21.98666667 -21.98666667 -21.98666667   0.           0.
   0.         -21.98666667   0.         -21.98666667 -21.98666667
 -21.98666667 -21.98666667   0.         -21.98666667 -21.98666667
 -21.98666667   0.           0.         -21.98666667 -21.98666667
 -21.98666667   0.           0.         -21.98666667 -21.98666667
 -21.98666667   0.           0.         -21.98666667 -21.98666667
   0.         -21.98666667   0.           0.           0.
 -21.98666667 -21.98666667 -21.98666667   0.           0.
 -21.98666667 -21.98666667 -21.98666667   0.         -21.98666667


 17%|█▋        | 26/150 [00:06<00:32,  3.80it/s]


KeyboardInterrupt: 